## Add

In this tutorial, we will construct a 2-bit adder.

Since we are using the ICE40, we need to set the target of `Mantle` to `"ice40"`.

In [1]:
import magma as m
m.set_mantle_target("ice40")

In the last example, we created a Python function that created a full adder. 
In this example, we are going to use the built-in `FullAdder` from `Mantle`, our standard library of useful circuits.

In [2]:
from mantle import FullAdder

import lattice ice40
import lattice mantle40


We can print out the *interface* of the `FullAdder`.

In [3]:
print('{} [{}]'.format(str(FullAdder), str(FullAdder.interface)))

FullAdder ["I0", In(Bit), "I1", In(Bit), "CIN", In(Bit), "O", Out(Bit), "COUT", Out(Bit)]


This tells us that the full adder has three inputs `I0`, `I1`, and `CIN`. Note that the type of these arguments are `In(Bit)`. 
In `Magma` all arguments on an interface must be *qualified* to be inputs or outputs. The function `In` convert a `Magma` type to an input. There are also two outputs `O` and `COUT`, both with type `Out(Bit)`.

Now let's build a 2-bit adder using `FullAdder`. 
We'll use a simple ripple carry adder design by connecting the carry out of one full adder
to the carry in of the next full adder.
The resulting adder will accept as input a carry in,
and generate a final carry out. Here's a diagram of the circuit we will construct:

![2-Bit Adder](logisim/adder.png)

Here is a Python function that constructs the adder.

In [4]:
def adder(A, B, Cin):
    n = len(A)
    C = []
    Cout = Cin
    for i in range(n):
        fulladder = FullAdder()
        m.wire(A[i], fulladder.I0)
        m.wire(B[i], fulladder.I1)
        m.wire(Cout, fulladder.CIN)
        C.append(fulladder.O)
        Cout = fulladder.COUT
    return m.uint(C), Cout 

We construct multiple instances of the full adder by calling `FullAdder()` inside the `for` loop. We also create a list of single bit outputs in the Python variable `C`. 
We then return this list converted to a `Magma` `Uint`. In addition to `Bits(n)`,
`Magma` also has built in types `UInt(n)` and `SInt(n)` to represent unsigned and signed ints.
`Magma` also has type conversion functions `bits`, `uint`, and `sint` to convert
between different types. 
In this example, `m.uint(C)` converts the list of bits to a `UInt(len(C))`.

In order to test the adder,
we setup the IceStick board
to have four inputs and 2 outputs.
As before, `J1` will be used for inputs and `J3` for outputs.

In [5]:
N = 2

from loam.boards.icestick import IceStick

icestick = IceStick()
for i in range(N):
    icestick.J1[i].input().on()
    icestick.J1[i+N].input().on()
    
for i in range(N+1):
    icestick.J3[i].output().on()

We define a `main` function that instances our 2-bit adder and wires it up to `J1` and `J3`. Notice the use of Python's slicing syntax using our width variable `N`.

In [6]:
main = icestick.DefineMain()

_sum, cout = adder(main.J1[0:N], main.J1[N:N+N], 0)
m.wire( _sum, main.J3[:N] )
m.wire( cout, main.J3[N] )

m.EndDefine()

As before, we compile with the `"lattice"` vendor.

In [7]:
m.compile('build/add', main)

compiling FullAdder
compiling main


And use our `yosys`, `arcachne-pnr`, and `icestorm` tool flow.

In [8]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif add.blif' add.v
arachne-pnr -q -d 1k -o add.txt -p add.pcf add.blif 
icepack add.txt add.bin
iceprog add.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build


init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x12 0x67 0x21 0x20 0x00 0x61 0x00 0x53 0x04 0x11 0x11 0x03 0xB1
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


You can test the program by connecting up some switches and LEDs to the headers. You should see the sum of the inputs displayed on the LEDs. First, we need to find out what pins `J1` and `J3` are wired up to. (Note: you can use `%` to execute shell commands inline in Jupyter notebooks)

In [9]:
%cat build/add.pcf

set_io J1[0] 112
set_io J1[1] 113
set_io J1[2] 114
set_io J1[3] 115
set_io J3[2] 60
set_io J3[1] 61
set_io J3[0] 62


In this example, we have J1 wire up to the four switch/LED circuits on the left, and J3 wired up to the three LED (no switch) circuits on the right
![](images/adder-seq/IMG_20180610_182758.jpg)
![](images/adder-seq/IMG_20180610_182802.jpg)
![](images/adder-seq/IMG_20180610_182806.jpg)
![](images/adder-seq/IMG_20180610_182811.jpg)
![](images/adder-seq/IMG_20180610_182819.jpg)
![](images/adder-seq/IMG_20180610_182823.jpg)
![](images/adder-seq/IMG_20180610_182827.jpg)

Again, it can be useful to examine the compiled Verilog.

Notice that it includes a Verilog definition of the `mantle` `FullAdder` implemented using the `SB_LUT4` and `SB_CARRY` primtives. The `main` module instances two `FullAdder`s and wires them up. It's quite simple.

In [10]:
%cat build/add.v

module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I0), .I1(I1), .I2(CIN), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0), .I1(I1), .CI(CIN), .CO(inst1_CO));
assign O = inst0_O;
assign COUT = inst1_CO;
endmodule

module main (input [3:0] J1, output [2:0] J3);
wire  inst0_O;
wire  inst0_COUT;
wire  inst1_O;
wire  inst1_COUT;
FullAdder inst0 (.I0(J1[0]), .I1(J1[2]), .CIN(1'b0), .O(inst0_O), .COUT(inst0_COUT));
FullAdder inst1 (.I0(J1[1]), .I1(J1[3]), .CIN(inst0_COUT), .O(inst1_O), .COUT(inst1_COUT));
assign J3 = {inst1_COUT,inst1_O,inst0_O};
endmodule

